### 時間序列新增col
    count(沒有的補0) 

In [1]:
import time
import pandas as pd
import datetime
import os
import itertools
import calendar

In [2]:
# 檔案讀取位置
datasite = './TimeSeriesData/TS3-201902_202306.parquet'

# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'./TimeSeriesData'
DataName = 'TS4-2022'


# 设置要補滿的起始年份、月份和结束年份、月份(多設定一個月forMA，最後再排除)
start_year = 2021
start_month = 12
end_year = 2023
end_month = 1

#輸出月份
output_start_day_str = '2022-01-01'  #含這天
output_end_day_str   = '2023-01-01'  #不含這天

#-------------------------------------------------------------------------------------------------#
output_start_day = datetime.datetime.strptime(output_start_day_str, '%Y-%m-%d')
output_end_day = datetime.datetime.strptime(output_end_day_str, '%Y-%m-%d')

# 記錄開始時間
start_time = time.time()

df = pd.read_parquet(datasite)

# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time

print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())
df

程式執行花費的時間： 1.79 秒
done時間: 2023-08-31 12:26:44.753569


,Name,year,month,day,hour,weekday,PULocationID,is_holiday,count,lat,lon,7DaysMA
0,lyft,2019,2,1,0,4,3,False,4,40.864294,-73.846510,170.285714
1,lyft,2019,2,1,0,4,4,False,22,40.723853,-73.975209,707.571429
2,lyft,2019,2,1,0,4,6,False,1,40.601117,-74.071747,50.714286
3,lyft,2019,2,1,0,4,7,False,50,40.761433,-73.919365,1640.857143
4,lyft,2019,2,1,0,4,9,False,2,40.750302,-73.790188,80.285714
...,...,...,...,...,...,...,...,...,...,...,...,...
22870509,yellow,2023,6,30,23,4,255,False,7,40.719651,-73.961672,50.142857
22870510,yellow,2023,6,30,23,4,256,False,7,40.710736,-73.962940,39.428571
22870511,yellow,2023,6,30,23,4,261,False,19,40.707456,-74.013983,532.857143
22870512,yellow,2023,6,30,23,4,262,False,18,40.778363,-73.943489,1216.714286


In [3]:
# 記錄開始時間
start_time = time.time()

# 创建所有可能的组合
years = range(start_year, end_year + 1)
months = range(1,13)
location_ids = df['PULocationID'].unique()  # 用您的位置ID列表替换
names = ['lyft','uber','yellow']
print('地點數量:',df['PULocationID'].nunique())

combinations = []

for name,year, month, location_id in itertools.product(names,years, months, location_ids):
    if start_year == end_year :
        if start_month <= month <= end_month:
            days_in_month = calendar.monthrange(year, month)[1]
            days = range(1, days_in_month + 1)
            hours = range(24)
            for day, hour in itertools.product(days, hours):
                combinations.append((name,year, month, day, hour, location_id))
    elif  start_year < end_year:
        if year == start_year:
            if month >= start_month:
                days_in_month = calendar.monthrange(year, month)[1]
                days = range(1, days_in_month + 1)
                hours = range(24)
                for day, hour in itertools.product(days, hours):
                    combinations.append((name,year, month, day, hour, location_id))
        elif year > start_year:
            if year < end_year:
                days_in_month = calendar.monthrange(year, month)[1]
                days = range(1, days_in_month + 1)
                hours = range(24)
                for day, hour in itertools.product(days, hours):
                    combinations.append((name,year, month, day, hour, location_id))
            elif year == end_year:
                if month <= end_month:
                    days_in_month = calendar.monthrange(year, month)[1]
                    days = range(1, days_in_month + 1)
                    hours = range(24)
                    for day, hour in itertools.product(days, hours):
                        combinations.append((name,year, month, day, hour, location_id))


# 创建数据框
columns = ["Name","year", "month", "day", "hour", "PULocationID"]
df_total = pd.DataFrame(combinations, columns=columns)

# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

df_total

地點數量: 261

程式執行花費的時間： 33.87 秒
done時間: 2023-08-31 12:27:18.666555


,Name,year,month,day,hour,PULocationID
0,lyft,2021,12,1,0,3
1,lyft,2021,12,1,1,3
2,lyft,2021,12,1,2,3
3,lyft,2021,12,1,3,3
4,lyft,2021,12,1,4,3
...,...,...,...,...,...,...
8024179,yellow,2023,1,31,19,199
8024180,yellow,2023,1,31,20,199
8024181,yellow,2023,1,31,21,199
8024182,yellow,2023,1,31,22,199


In [4]:
#節日資訊
import numpy as np
from holidays.countries import US
from holidays.calendars import gregorian
from holidays.calendars.gregorian import _get_nth_weekday_of_month
class AllHolidays(US):
    def _populate(self, year):
        super()._populate(year)
        self.pop_named("Lincoln's Birthday")
        self.pop_named("Susan B. Anthony Day")
        
        self._add_holiday("Valentine's Day", 2, 14)
        self._add_holiday("St.Patrick's Day", 3, 17)
        self._add_holiday("Halloween", 10, 31)
     
        self._add_holiday("Christmas Eve", 12, 24)
        self._add_holiday("Christmas season", 12, 26)
        self._add_holiday("Christmas season", 12, 27)
        self._add_holiday("Christmas season", 12, 28)
        self._add_holiday("Christmas season", 12, 29)
        self._add_holiday("Christmas season", 12, 30)
        
        self._add_holiday("New Year's Eve", 12, 31)

# 實例化AllHolidays
ny_holidays_all = AllHolidays(subdiv='NY', years=[2019, 2020, 2021, 2022, 2023])

# 新增2019~2023年的復活節、復活節星期一
ny_holidays_all[datetime.date(2019, 4, 21)] = "Easter"
ny_holidays_all[datetime.date(2019, 4, 22)] = "Easter Monday"
ny_holidays_all[datetime.date(2020, 4, 12)] = "Easter"
ny_holidays_all[datetime.date(2020, 4, 13)] = "Easter Monday"
ny_holidays_all[datetime.date(2021, 4, 4)] = "Easter"
ny_holidays_all[datetime.date(2021, 4, 5)] = "Easter Monday"
ny_holidays_all[datetime.date(2022, 4, 17)] = "Easter"
ny_holidays_all[datetime.date(2022, 4, 18)] = "Easter Monday"
ny_holidays_all[datetime.date(2023, 4, 9)] = "Easter"
ny_holidays_all[datetime.date(2023, 4, 10)] = "Easter Monday"

# 新增2019~2023感恩節
ny_holidays_all[datetime.date(2019,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 29)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 30)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 26)] =  "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 23)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 25)] = "Thanksgiving weekend"



In [5]:
#地點經緯度
datasite_temp = '../taxi_zone_lat_lon/organized_taxi_zone_lat_lon.csv'
#創立一個 df_temp 讀經緯度資料
df_temp = pd.read_csv(datasite_temp)

df_temp

,zone,LocationID,borough,lat,lon
0,Newark Airport,1,EWR,40.690243,-74.174270
1,Jamaica Bay,2,Queens,40.612163,-73.817643
2,Allerton/Pelham Gardens,3,Bronx,40.864294,-73.846510
3,Alphabet City,4,Manhattan,40.723853,-73.975209
4,Arden Heights,5,Staten Island,40.556678,-74.189803
...,...,...,...,...,...
258,Williamsburg (South Side),256,Brooklyn,40.710736,-73.962940
259,Woodlawn/Wakefield,259,Bronx,40.900107,-73.853635
260,Woodside,260,Queens,40.746439,-73.905907
261,World Trade Center,261,Manhattan,40.707456,-74.013983


In [6]:
# 記錄開始時間
start_time = time.time()

df = df_total.merge(df[['Name','year', 'month','day','hour','PULocationID','weekday','is_holiday','count','lat','lon','7DaysMA']], on=['Name','year', 'month','day','hour','PULocationID'], how='left')
del df_total
# 臨時日期
df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str),format='%Y-%m-%d')
# weekday
df['weekday'] = (pd.to_datetime(df['TempTime'])).dt.weekday
# is_holiday
df['is_holiday'] = np.where(df['TempTime'].dt.date.isin(ny_holidays_all), True, False)
df['is_holiday'].value_counts()
df['count'] = df['count'].fillna(0).astype(int)
#經緯度
df = df.drop(['lat'],axis = 1)
df = df.drop(['lon'],axis = 1)
df = df.merge(df_temp[['LocationID', 'lat','lon']], left_on=['PULocationID'], right_on=['LocationID'], how='left')
df = df.drop(['LocationID'],axis = 1)
del df_temp 
#MA
N = 3 #往前天數
M = 3 #往後天數
df = df.drop(['7DaysMA'],axis = 1)
df_temp = df[['Name','PULocationID','TempTime','count']]
df_temp = df_temp.groupby([ 'Name','TempTime','PULocationID'])['count'].sum().reset_index()
TempTime_unique_count = df['TempTime'].nunique()
print(f'總天數:{TempTime_unique_count}')
temp = 0
temp1 = '0'
for index, row in df_temp.iterrows():
    temp_start_time = row['TempTime'] - pd.DateOffset(days=N)
    temp_end_time = row['TempTime'] + pd.DateOffset(days=M)
    temp_count = df_temp.loc[(df_temp['TempTime'] >= temp_start_time) & (df_temp['TempTime'] <= temp_end_time) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['count'].sum()
    temp_day =  row['TempTime'] - df_temp['TempTime'].min()
    if row['TempTime'].day == 1:  # 每個月的1日印出來
        if (temp != row['TempTime']) | (temp1  != row['Name']) :
            temp = row['TempTime']
            temp1 = row['Name']
            print(row['Name'],row['TempTime'])
    if TempTime_unique_count - temp_day.days <= M:          
        temp_count =  df_temp.loc[(df_temp['TempTime'] >= df_temp['TempTime'].max() -pd.DateOffset(days=N)-pd.DateOffset(days=M)) & (df_temp['TempTime'] <= df_temp['TempTime'].max() ) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['count'].sum()
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
    elif temp_day.days >= N:  
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
    else : #處理前N-1天
        temp_count =  df_temp.loc[(df_temp['TempTime'] >= df_temp['TempTime'].min()) & (df_temp['TempTime'] <= ( df_temp['TempTime'].min() +pd.DateOffset(days=N)+pd.DateOffset(days=M)) ) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['count'].sum()
        #temp_count = temp_count *(N+1) / (temp_day.days+1)        
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count      
del temp_start_time,temp_end_time,temp_count,temp_day,temp,temp1  

# 放回 df
merged_df = df.merge(df_temp[['Name', 'TempTime','PULocationID', f'{N+1+M}DaysTimes']], on=['Name', 'TempTime','PULocationID'], how='left')
df[f'{N+1+M}DaysTimes'] = merged_df[f'{N+1+M}DaysTimes']
del merged_df

#留下需求月份
df = df[(df['TempTime'] >= output_start_day) &  (df['TempTime'] < output_end_day) ]
# 編號
df = df.sort_index(ignore_index=True)

# TempTime移除
df = df.drop(columns=['TempTime'])


# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/{DataName}.parquet')
    print('save!')
    
# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

總天數:427
lyft 2021-12-01 00:00:00
lyft 2022-01-01 00:00:00
lyft 2022-02-01 00:00:00
lyft 2022-03-01 00:00:00
lyft 2022-04-01 00:00:00
lyft 2022-05-01 00:00:00
lyft 2022-06-01 00:00:00
lyft 2022-07-01 00:00:00
lyft 2022-08-01 00:00:00
lyft 2022-09-01 00:00:00
lyft 2022-10-01 00:00:00
lyft 2022-11-01 00:00:00
lyft 2022-12-01 00:00:00
lyft 2023-01-01 00:00:00
uber 2021-12-01 00:00:00
uber 2022-01-01 00:00:00
uber 2022-02-01 00:00:00
uber 2022-03-01 00:00:00
uber 2022-04-01 00:00:00
uber 2022-05-01 00:00:00
uber 2022-06-01 00:00:00
uber 2022-07-01 00:00:00
uber 2022-08-01 00:00:00
uber 2022-09-01 00:00:00
uber 2022-10-01 00:00:00
uber 2022-11-01 00:00:00
uber 2022-12-01 00:00:00
uber 2023-01-01 00:00:00
yellow 2021-12-01 00:00:00
yellow 2022-01-01 00:00:00
yellow 2022-02-01 00:00:00
yellow 2022-03-01 00:00:00
yellow 2022-04-01 00:00:00
yellow 2022-05-01 00:00:00
yellow 2022-06-01 00:00:00
yellow 2022-07-01 00:00:00
yellow 2022-08-01 00:00:00
yellow 2022-09-01 00:00:00
yellow 2022-10-01 00:0

In [9]:
df

,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,LocationID,lat,lon,7DaysTimes
0,lyft,2022,1,1,0,3,5,True,16,3,40.864294,-73.846510,1760.0
1,lyft,2022,1,1,1,3,5,True,21,3,40.864294,-73.846510,1760.0
2,lyft,2022,1,1,2,3,5,True,6,3,40.864294,-73.846510,1760.0
3,lyft,2022,1,1,3,3,5,True,11,3,40.864294,-73.846510,1760.0
4,lyft,2022,1,1,4,3,5,True,10,3,40.864294,-73.846510,1760.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859075,yellow,2022,12,31,19,199,5,True,0,199,40.793489,-73.880683,0.0
6859076,yellow,2022,12,31,20,199,5,True,0,199,40.793489,-73.880683,0.0
6859077,yellow,2022,12,31,21,199,5,True,0,199,40.793489,-73.880683,0.0
6859078,yellow,2022,12,31,22,199,5,True,0,199,40.793489,-73.880683,0.0
